In [1]:
# import common packages
import paths
import numpy as np

import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

# lib from QISChem
from qiskit_acqua_chemistry import FermionicOperator

# lib from optimizer and algorithm
from qiskit_acqua.operator import Operator
from qiskit_acqua import (get_algorithm_instance, get_optimizer_instance, get_variational_form_instance)

# lib for driver
from qiskit_acqua_chemistry.drivers import ConfigurationManager
from collections import OrderedDict

In [2]:
# using driver to get fermionic Hamiltonian
# PySCF example
cfg_mgr = ConfigurationManager()
pyscf_cfg = OrderedDict([('atom', 'H .0 .0 .0; H .0 .0 0.735'), ('unit', 'Angstrom'), ('charge', 0), ('spin', 0), ('basis', 'sto3g')])
section = {}
section['properties'] = pyscf_cfg
driver = cfg_mgr.get_driver_instance('PYSCF')
molecule = driver.run(section)
print(molecule._hf_energy)
print(molecule._nuclear_repulsion_energy)

print(molecule._hf_energy - molecule._nuclear_repulsion_energy)

-1.1169989967540044
0.7199689944489797
-1.8369679912029842


In [3]:
# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule._one_body_integrals, h2=molecule._two_body_integrals)
map_type='JORDAN_WIGNER'
qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp.convert('paulis', 'matrix')
# print(qubitOp.print_operators())
statevector = ferOp.init_hartree_fock_state(map_type, 'matrix', 2, 4)

exact_eigensolver = get_algorithm_instance('ExactEigensolver')
exact_eigensolver.init_args(qubitOp, k=1)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret['energy']))

# qubitOp.eval('matrix', input_circuit, num_shots, backend, quantum_program):)
# print(qubitOp.matrix)
# print(statevector.qasm())
print(statevector.T.conj().dot(qubitOp.matrix.dot(statevector)))

AttributeError: 'FermionicOperator' object has no attribute 'init_hartree_fock_state'

In [ ]:
# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule._one_body_integrals, h2=molecule._two_body_integrals)
map_type='PARITY'
qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp.convert('paulis', 'matrix')
# print(qubitOp.print_operators())
statevector = ferOp.init_hartree_fock_state(map_type, 'matrix', 2, 4)

exact_eigensolver = get_algorithm_instance('ExactEigensolver')
exact_eigensolver.init_args(qubitOp, k=1)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret['energy']))

# qubitOp.eval('matrix', input_circuit, num_shots, backend, quantum_program):)
# print(qubitOp.matrix)
print(statevector)
# print(statevector.qasm())
print(statevector.T.conj().dot(qubitOp.matrix.dot(statevector)))